# irodsConnector 

## Create an iRODS session (connection to iRODS server)

In [1]:
from ibridges.irodsconnector.session import Session
import os, json
from getpass import getpass

### Password authentication

In [2]:
with open(os.path.expanduser("~/.irods/irods_environment.json"), "r") as f:
    ienv = json.load(f)
password = getpass("Your iRODS password")
session = Session(irods_env=ienv, password=password)

Auth with password


### Using the cached password ~/.irods/.rodsA

In [3]:
session = Session(irods_env_path=os.path.expanduser("~/.irods/irods_environment.json"))

Auth with password


TypeError: expected string or bytes-like object, got 'NoneType'

### Checking some session parameters

In [4]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) # default home for iRODS /zone/home/username
print(session.server_version)

r.d.schram@uu.nl
irodsResc
nluu12p
(4, 2, 12)


## Metadata of data objects and collections
### Retrieve an iRODS object or collection and list its metadata

In [5]:
#print(ienv)
coll_path = ienv.get('irods_home', '') + '/books/BenHur.txt'
print(coll_path)
coll = session.irods_session.data_objects.get(coll_path) # TODO: exchange once data_ops is done

/nluu12p/home/research-test-christine/books/BenHur.txt


In [6]:
from ibridges.irodsconnector.meta import MetaData
coll_meta = MetaData(coll)
print(coll_meta)

 - {name: AUTHOR, value: Lew Wallace, units: None}
 - {name: HUHU, value: hihi, units: None}
 - {name: NEWKEY, value: hihi, units: None}



### Add, set and delete metadata

In [7]:
coll_meta.add('NewKey', 'NewValue')
print(coll_meta)

 - {name: AUTHOR, value: Lew Wallace, units: None}
 - {name: NEWKEY, value: NewValue, units: None}
 - {name: HUHU, value: hihi, units: None}
 - {name: NEWKEY, value: hihi, units: None}



Note, that keys are always capitalised. This is good practice in iRODS.

In [8]:
coll_meta.delete('NEWKEY', 'NewValue')
print(coll_meta)

 - {name: AUTHOR, value: Lew Wallace, units: None}
 - {name: HUHU, value: hihi, units: None}
 - {name: NEWKEY, value: hihi, units: None}



We can also set the meta data to a single key, value, units pair. This will remove any other entries with the same key.

In [ ]:
coll_meta.set("NEWKEY", "OtherValue")

## Resources and handling resources

In [ ]:
from ibridges.irodsconnector.resources import Resources
resources = Resources(session)

### Check if default resource exists

In [ ]:
default_resc = resources.get_resource(session._irods_env.get("irods_default_resource", ""))
print(default_resc.name)
print(default_resc.free_space) # Metadata how much bytes are left on resource, None if not set
print(default_resc.type) # Storage policy

### Listing resources

In [ ]:
print(resources.resources()) # all resources
print(resources.root_resources) # all writeable resources (name, status, free space, context)

### Retrieve current free space
In contrast to `resc.free_space` the function `get_free_space` accumulates all free space in the subtree starting with the resource as parent.

In [ ]:
resources.get_free_space(session._irods_env.get("irods_default_resource", "")) # default resource name

## Tickets (access string to collection or data object)
### List all tickets which you issued

In [ ]:
from ibridges.irodsconnector.tickets import Tickets
tickets = Tickets(session)
print(tickets.all_ticket_strings)
print(tickets.all_tickets()) # (ticket string, access mode, object or collection id, expiry date in epoche)

### Issue a ticket

In [ ]:
from datetime import datetime

exp_date = datetime.today().strftime('%Y-%m-%d.%H:%M:%S')
objPath = "/nluu12p/home/research-test-christine/books/BenHur.txt"
ticket = tickets.create_ticket(obj_path=objPath, ticket_type="write", expiry_string=exp_date) # allow write access

### Fetch and delete a ticket

In [ ]:
ticket = tickets.get_ticket(tickets.all_ticket_strings[0])
tickets.delete_ticket(ticket)